In [71]:
import pandas as pd
import plotly.express as px
import requests
import duckdb
from pathlib import Path


In [72]:
data_path = Path('/content/data folder')

bronze_path = data_path / "bronze"
silver_path = data_path / "silver"
gold_path = data_path / "gold"

In [73]:


duckdb_path = data_path / "worldbank_poplulation_data.duckdb"

duck_conn = duckdb.connect(str(duckdb_path))

In [74]:
countries_url = 'http://api.worldbank.org/v2/country?format=json&per_page=400'

countries_response = requests.get(countries_url)
countries_json = countries_response.json()


country_rows = countries_json[1] if len(countries_json) > 1 else []

data_raw = pd.json_normalize(country_rows)


data_raw.head()


,id,iso2Code,name,capitalCity,longitude,latitude,region.id,region.iso2code,region.value,adminregion.id,adminregion.iso2code,adminregion.value,incomeLevel.id,incomeLevel.iso2code,incomeLevel.value,lendingType.id,lendingType.iso2code,lendingType.value
0,ABW,AW,Aruba,Oranjestad,-70.0167,12.5167,LCN,ZJ,Latin America & Caribbean,,,,HIC,XD,High income,LNX,XX,Not classified
1,AFE,ZH,Africa Eastern and Southern,,,,NA,NA,Aggregates,,,,NA,NA,Aggregates,,,Aggregates
2,AFG,AF,Afghanistan,Kabul,69.1761,34.5228,MEA,ZQ,"Middle East, North Africa, Afghanistan & Pakistan",MNA,XQ,"Middle East, North Africa, Afghanistan & Pakis...",LIC,XM,Low income,IDX,XI,IDA
3,AFR,A9,Africa,,,,NA,NA,Aggregates,,,,NA,NA,Aggregates,,,Aggregates
4,AFW,ZI,Africa Western and Central,,,,NA,NA,Aggregates,,,,NA,NA,Aggregates,,,Aggregates


In [75]:
data_filt = data_raw[data_raw['region.id'] !="NA" ].copy()

data_pick = data_filt[[
    "id",
    "iso2Code",
    "name",
    "capitalCity",
    "longitude",
    "latitude",
    "region.value",
    "incomeLevel.value"
]].rename(columns={
    "id":"iso3",
    "iso2Code":"iso2",
    "name":"Official_Name",
    "region.value":"region_name",
    "incomeLevel.value":"income_level",
    "capitalCity":"capital_city"
})

In [76]:
population_url = "https://api.worldbank.org/v2/country/all/indicator/SP.POP.TOTL?format=json&MRV=1&per_page=20000"

population_response = requests.get(population_url)
population_json = population_response.json()

population_rows = population_json[1] if len(population_json) > 1 else []

data_pop = pd.json_normalize(population_rows)

data_pop


,countryiso3code,date,value,unit,obs_status,decimal,indicator.id,indicator.value,country.id,country.value
0,AFE,2024,769294618.0,,,0,SP.POP.TOTL,"Population, total",ZH,Africa Eastern and Southern
1,AFW,2024,521764076.0,,,0,SP.POP.TOTL,"Population, total",ZI,Africa Western and Central
2,ARB,2024,492579811.0,,,0,SP.POP.TOTL,"Population, total",1A,Arab World
3,CSS,2024,4539060.0,,,0,SP.POP.TOTL,"Population, total",S3,Caribbean small states
4,CEB,2024,100050066.0,,,0,SP.POP.TOTL,"Population, total",B8,Central Europe and the Baltics
...,...,...,...,...,...,...,...,...,...,...
261,VIR,2024,104377.0,,,0,SP.POP.TOTL,"Population, total",VI,Virgin Islands (U.S.)
262,PSE,2024,5289152.0,,,0,SP.POP.TOTL,"Population, total",PS,West Bank and Gaza
263,YEM,2024,40583164.0,,,0,SP.POP.TOTL,"Population, total",YE,"Yemen, Rep."
264,ZMB,2024,21314956.0,,,0,SP.POP.TOTL,"Population, total",ZM,Zambia


In [77]:
data_pop_clean = data_pop[["countryiso3code", "date", "value"]].rename(columns={
    "countryiso3code":"iso3",
    "date":"population_year",
    "value":"population"
})

data_pop_filt = data_pop_clean.dropna(subset=["iso3"])

In [78]:
data_pop_fixed = data_pop_filt.assign(

      population = pd.to_numeric(data_pop_filt["population"], errors= "coerce"),
      population_year = pd.to_numeric(data_pop_filt["population_year"], errors= "coerce")

).dropna(subset=["population","population_year"])

In [79]:
bronze_df = data_pick.merge(data_pop_fixed, on="iso3", how="left" )
bronze_df["scrape_timesstamp"] = pd.Timestamp.utcnow()

bronze_df

,iso3,iso2,Official_Name,capital_city,longitude,latitude,region_name,income_level,population_year,population,scrape_timesstamp
0,ABW,AW,Aruba,Oranjestad,-70.0167,12.5167,Latin America & Caribbean,High income,2024,107624.0,2025-11-12 20:41:08.364617+00:00
1,AFG,AF,Afghanistan,Kabul,69.1761,34.5228,"Middle East, North Africa, Afghanistan & Pakistan",Low income,2024,42647492.0,2025-11-12 20:41:08.364617+00:00
2,AGO,AO,Angola,Luanda,13.242,-8.81155,Sub-Saharan Africa,Lower middle income,2024,37885849.0,2025-11-12 20:41:08.364617+00:00
3,ALB,AL,Albania,Tirane,19.8172,41.3317,Europe & Central Asia,Upper middle income,2024,2714617.0,2025-11-12 20:41:08.364617+00:00
4,AND,AD,Andorra,Andorra la Vella,1.5218,42.5075,Europe & Central Asia,High income,2024,81938.0,2025-11-12 20:41:08.364617+00:00
...,...,...,...,...,...,...,...,...,...,...,...
212,XKX,XK,Kosovo,Pristina,20.926,42.565,Europe & Central Asia,Upper middle income,2024,1527324.0,2025-11-12 20:41:08.364617+00:00
213,YEM,YE,"Yemen, Rep.",Sana'a,44.2075,15.352,"Middle East, North Africa, Afghanistan & Pakistan",Low income,2024,40583164.0,2025-11-12 20:41:08.364617+00:00
214,ZAF,ZA,South Africa,Pretoria,28.1871,-25.746,Sub-Saharan Africa,Upper middle income,2024,64007187.0,2025-11-12 20:41:08.364617+00:00
215,ZMB,ZM,Zambia,Lusaka,28.2937,-15.3982,Sub-Saharan Africa,Lower middle income,2024,21314956.0,2025-11-12 20:41:08.364617+00:00


In [80]:
bronze = bronze_path / "country_population_bronze.csv"
bronze_df.to_csv(bronze, index=False)

In [81]:
duck_conn.execute("CREATE SCHEMA IF NOT EXISTS bronze")
duck_conn.register("bronze_table", bronze_df )
duck_conn.execute("CREATE OR REPLACE TABLE bronze.population AS SELECT * FROM bronze_table")
duck_conn.unregister("bronze_table")


In [82]:
bronze_pr = duck_conn.execute("SELECT * FROM bronze.population LIMIT 5").df()

In [83]:
bronze_pr

,iso3,iso2,Official_Name,capital_city,longitude,latitude,region_name,income_level,population_year,population,scrape_timesstamp
0,ABW,AW,Aruba,Oranjestad,-70.0167,12.5167,Latin America & Caribbean,High income,2024,107624.0,2025-11-12 20:41:08.364617+00:00
1,AFG,AF,Afghanistan,Kabul,69.1761,34.5228,"Middle East, North Africa, Afghanistan & Pakistan",Low income,2024,42647492.0,2025-11-12 20:41:08.364617+00:00
2,AGO,AO,Angola,Luanda,13.242,-8.81155,Sub-Saharan Africa,Lower middle income,2024,37885849.0,2025-11-12 20:41:08.364617+00:00
3,ALB,AL,Albania,Tirane,19.8172,41.3317,Europe & Central Asia,Upper middle income,2024,2714617.0,2025-11-12 20:41:08.364617+00:00
4,AND,AD,Andorra,Andorra la Vella,1.5218,42.5075,Europe & Central Asia,High income,2024,81938.0,2025-11-12 20:41:08.364617+00:00


In [84]:
bronze_df_data = duck_conn.execute("SELECT * FROM bronze.population").df()
bronze_df_data

,iso3,iso2,Official_Name,capital_city,longitude,latitude,region_name,income_level,population_year,population,scrape_timesstamp
0,ABW,AW,Aruba,Oranjestad,-70.0167,12.5167,Latin America & Caribbean,High income,2024,107624.0,2025-11-12 20:41:08.364617+00:00
1,AFG,AF,Afghanistan,Kabul,69.1761,34.5228,"Middle East, North Africa, Afghanistan & Pakistan",Low income,2024,42647492.0,2025-11-12 20:41:08.364617+00:00
2,AGO,AO,Angola,Luanda,13.242,-8.81155,Sub-Saharan Africa,Lower middle income,2024,37885849.0,2025-11-12 20:41:08.364617+00:00
3,ALB,AL,Albania,Tirane,19.8172,41.3317,Europe & Central Asia,Upper middle income,2024,2714617.0,2025-11-12 20:41:08.364617+00:00
4,AND,AD,Andorra,Andorra la Vella,1.5218,42.5075,Europe & Central Asia,High income,2024,81938.0,2025-11-12 20:41:08.364617+00:00
...,...,...,...,...,...,...,...,...,...,...,...
212,XKX,XK,Kosovo,Pristina,20.926,42.565,Europe & Central Asia,Upper middle income,2024,1527324.0,2025-11-12 20:41:08.364617+00:00
213,YEM,YE,"Yemen, Rep.",Sana'a,44.2075,15.352,"Middle East, North Africa, Afghanistan & Pakistan",Low income,2024,40583164.0,2025-11-12 20:41:08.364617+00:00
214,ZAF,ZA,South Africa,Pretoria,28.1871,-25.746,Sub-Saharan Africa,Upper middle income,2024,64007187.0,2025-11-12 20:41:08.364617+00:00
215,ZMB,ZM,Zambia,Lusaka,28.2937,-15.3982,Sub-Saharan Africa,Lower middle income,2024,21314956.0,2025-11-12 20:41:08.364617+00:00


In [85]:
silver_step = bronze_df_data[bronze_df_data["population"].notna()].copy()

In [86]:
silver_step

,iso3,iso2,Official_Name,capital_city,longitude,latitude,region_name,income_level,population_year,population,scrape_timesstamp
0,ABW,AW,Aruba,Oranjestad,-70.0167,12.5167,Latin America & Caribbean,High income,2024,107624.0,2025-11-12 20:41:08.364617+00:00
1,AFG,AF,Afghanistan,Kabul,69.1761,34.5228,"Middle East, North Africa, Afghanistan & Pakistan",Low income,2024,42647492.0,2025-11-12 20:41:08.364617+00:00
2,AGO,AO,Angola,Luanda,13.242,-8.81155,Sub-Saharan Africa,Lower middle income,2024,37885849.0,2025-11-12 20:41:08.364617+00:00
3,ALB,AL,Albania,Tirane,19.8172,41.3317,Europe & Central Asia,Upper middle income,2024,2714617.0,2025-11-12 20:41:08.364617+00:00
4,AND,AD,Andorra,Andorra la Vella,1.5218,42.5075,Europe & Central Asia,High income,2024,81938.0,2025-11-12 20:41:08.364617+00:00
...,...,...,...,...,...,...,...,...,...,...,...
212,XKX,XK,Kosovo,Pristina,20.926,42.565,Europe & Central Asia,Upper middle income,2024,1527324.0,2025-11-12 20:41:08.364617+00:00
213,YEM,YE,"Yemen, Rep.",Sana'a,44.2075,15.352,"Middle East, North Africa, Afghanistan & Pakistan",Low income,2024,40583164.0,2025-11-12 20:41:08.364617+00:00
214,ZAF,ZA,South Africa,Pretoria,28.1871,-25.746,Sub-Saharan Africa,Upper middle income,2024,64007187.0,2025-11-12 20:41:08.364617+00:00
215,ZMB,ZM,Zambia,Lusaka,28.2937,-15.3982,Sub-Saharan Africa,Lower middle income,2024,21314956.0,2025-11-12 20:41:08.364617+00:00


In [87]:
silver_step = silver_step[silver_step["population"] > 0]

In [88]:
silver_step

,iso3,iso2,Official_Name,capital_city,longitude,latitude,region_name,income_level,population_year,population,scrape_timesstamp
0,ABW,AW,Aruba,Oranjestad,-70.0167,12.5167,Latin America & Caribbean,High income,2024,107624.0,2025-11-12 20:41:08.364617+00:00
1,AFG,AF,Afghanistan,Kabul,69.1761,34.5228,"Middle East, North Africa, Afghanistan & Pakistan",Low income,2024,42647492.0,2025-11-12 20:41:08.364617+00:00
2,AGO,AO,Angola,Luanda,13.242,-8.81155,Sub-Saharan Africa,Lower middle income,2024,37885849.0,2025-11-12 20:41:08.364617+00:00
3,ALB,AL,Albania,Tirane,19.8172,41.3317,Europe & Central Asia,Upper middle income,2024,2714617.0,2025-11-12 20:41:08.364617+00:00
4,AND,AD,Andorra,Andorra la Vella,1.5218,42.5075,Europe & Central Asia,High income,2024,81938.0,2025-11-12 20:41:08.364617+00:00
...,...,...,...,...,...,...,...,...,...,...,...
212,XKX,XK,Kosovo,Pristina,20.926,42.565,Europe & Central Asia,Upper middle income,2024,1527324.0,2025-11-12 20:41:08.364617+00:00
213,YEM,YE,"Yemen, Rep.",Sana'a,44.2075,15.352,"Middle East, North Africa, Afghanistan & Pakistan",Low income,2024,40583164.0,2025-11-12 20:41:08.364617+00:00
214,ZAF,ZA,South Africa,Pretoria,28.1871,-25.746,Sub-Saharan Africa,Upper middle income,2024,64007187.0,2025-11-12 20:41:08.364617+00:00
215,ZMB,ZM,Zambia,Lusaka,28.2937,-15.3982,Sub-Saharan Africa,Lower middle income,2024,21314956.0,2025-11-12 20:41:08.364617+00:00


In [89]:
silver_clean = silver_step.assign(

      population=silver_step["population"].astype("Int64"),
      population_year=silver_step["population_year"].astype("Int64"),
      population_in_millions=silver_step["population"]/1000000
)

In [90]:
silver_csv = silver_path / "country_population_silver.csv"
silver_clean.to_csv(silver_csv, index=False)

In [91]:
duck_conn.execute("CREATE SCHEMA IF NOT EXISTS silver")
duck_conn.register("silver_table", silver_clean )
duck_conn.execute("CREATE OR REPLACE TABLE silver.population AS SELECT * FROM silver_table")
duck_conn.unregister("silver_table")


In [92]:
silver_from_db = duck_conn.execute("SELECT * FROM silver.population").df()

In [93]:
gold_sorted = silver_from_db.sort_values(
    ["iso3", "population_year","scrape_timesstamp"],
    ascending = [True, False, False]
)

In [94]:
gold_latest = gold_sorted.drop_duplicates(subset=["iso3"], keep="first").copy()


In [95]:
gold_final = gold_latest.assign(
    global_population_share = gold_latest["population"]/gold_latest["population"].sum(),
    population_rank = gold_latest["population"].rank(method = "dense" , ascending=False).astype(int)
)

In [96]:
gold_final

,iso3,iso2,Official_Name,capital_city,longitude,latitude,region_name,income_level,population_year,population,scrape_timesstamp,population_in_millions,global_population_share,population_rank
0,ABW,AW,Aruba,Oranjestad,-70.0167,12.5167,Latin America & Caribbean,High income,2024,107624,2025-11-12 20:41:08.364617+00:00,0.107624,0.000013,191
1,AFG,AF,Afghanistan,Kabul,69.1761,34.5228,"Middle East, North Africa, Afghanistan & Pakistan",Low income,2024,42647492,2025-11-12 20:41:08.364617+00:00,42.647492,0.005253,36
2,AGO,AO,Angola,Luanda,13.242,-8.81155,Sub-Saharan Africa,Lower middle income,2024,37885849,2025-11-12 20:41:08.364617+00:00,37.885849,0.004667,40
3,ALB,AL,Albania,Tirane,19.8172,41.3317,Europe & Central Asia,Upper middle income,2024,2714617,2025-11-12 20:41:08.364617+00:00,2.714617,0.000334,142
4,AND,AD,Andorra,Andorra la Vella,1.5218,42.5075,Europe & Central Asia,High income,2024,81938,2025-11-12 20:41:08.364617+00:00,0.081938,0.000010,197
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,XKX,XK,Kosovo,Pristina,20.926,42.565,Europe & Central Asia,Upper middle income,2024,1527324,2025-11-12 20:41:08.364617+00:00,1.527324,0.000188,153
213,YEM,YE,"Yemen, Rep.",Sana'a,44.2075,15.352,"Middle East, North Africa, Afghanistan & Pakistan",Low income,2024,40583164,2025-11-12 20:41:08.364617+00:00,40.583164,0.004999,38
214,ZAF,ZA,South Africa,Pretoria,28.1871,-25.746,Sub-Saharan Africa,Upper middle income,2024,64007187,2025-11-12 20:41:08.364617+00:00,64.007187,0.007884,24
215,ZMB,ZM,Zambia,Lusaka,28.2937,-15.3982,Sub-Saharan Africa,Lower middle income,2024,21314956,2025-11-12 20:41:08.364617+00:00,21.314956,0.002625,62


In [97]:
gold_csv = gold_path / "country_population_gold.csv"
gold_final.to_csv(gold_csv, index=False)

In [98]:
duck_conn.execute("CREATE SCHEMA IF NOT EXISTS gold")
duck_conn.register("gold_table", gold_final )
duck_conn.execute("CREATE OR REPLACE TABLE gold.population_latest AS SELECT * FROM gold_table")
duck_conn.unregister("gold_table")


In [99]:
gold_top10 = gold_final.sort_values("population", ascending=False).head(10)
print(gold_top10[["Official_Name" , "iso3" , "population" , "population_rank"]])

          Official_Name iso3  population  population_rank
89                India  IND  1450935791                1
36                China  CHN  1408975000                2
203       United States  USA   340110988                3
87            Indonesia  IDN   283487931                4
150            Pakistan  PAK   251269164                5
142             Nigeria  NGA   232679478                6
26               Brazil  BRA   211998573                7
17           Bangladesh  BGD   173562364                8
165  Russian Federation  RUS   143533851                9
61             Ethiopia  ETH   132059767               10


In [100]:
gold_data = duck_conn.execute("SELECT * FROM gold.population_latest").df()

In [101]:
gold_data

,iso3,iso2,Official_Name,capital_city,longitude,latitude,region_name,income_level,population_year,population,scrape_timesstamp,population_in_millions,global_population_share,population_rank
0,ABW,AW,Aruba,Oranjestad,-70.0167,12.5167,Latin America & Caribbean,High income,2024,107624,2025-11-12 20:41:08.364617+00:00,0.107624,0.000013,191
1,AFG,AF,Afghanistan,Kabul,69.1761,34.5228,"Middle East, North Africa, Afghanistan & Pakistan",Low income,2024,42647492,2025-11-12 20:41:08.364617+00:00,42.647492,0.005253,36
2,AGO,AO,Angola,Luanda,13.242,-8.81155,Sub-Saharan Africa,Lower middle income,2024,37885849,2025-11-12 20:41:08.364617+00:00,37.885849,0.004667,40
3,ALB,AL,Albania,Tirane,19.8172,41.3317,Europe & Central Asia,Upper middle income,2024,2714617,2025-11-12 20:41:08.364617+00:00,2.714617,0.000334,142
4,AND,AD,Andorra,Andorra la Vella,1.5218,42.5075,Europe & Central Asia,High income,2024,81938,2025-11-12 20:41:08.364617+00:00,0.081938,0.000010,197
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,XKX,XK,Kosovo,Pristina,20.926,42.565,Europe & Central Asia,Upper middle income,2024,1527324,2025-11-12 20:41:08.364617+00:00,1.527324,0.000188,153
213,YEM,YE,"Yemen, Rep.",Sana'a,44.2075,15.352,"Middle East, North Africa, Afghanistan & Pakistan",Low income,2024,40583164,2025-11-12 20:41:08.364617+00:00,40.583164,0.004999,38
214,ZAF,ZA,South Africa,Pretoria,28.1871,-25.746,Sub-Saharan Africa,Upper middle income,2024,64007187,2025-11-12 20:41:08.364617+00:00,64.007187,0.007884,24
215,ZMB,ZM,Zambia,Lusaka,28.2937,-15.3982,Sub-Saharan Africa,Lower middle income,2024,21314956,2025-11-12 20:41:08.364617+00:00,21.314956,0.002625,62


In [102]:
region_summary = gold_data.groupby("region_name").agg(
    total_population = ("population", "sum"),
    average_population = ("population", "mean"),
    countries = ("iso3" ,"count")
).reset_index().sort_values("total_population", ascending=False)

In [103]:
region_summary

,region_name,total_population,average_population,countries
0,East Asia & Pacific,2364965592,6.391799e+07,37
5,South Asia,1677384532,2.795641e+08,6
6,Sub-Saharan Africa,1291058694,2.689706e+07,48
1,Europe & Central Asia,928465742,1.600803e+07,58
3,"Middle East, North Africa, Afghanistan & Pakistan",813118814,3.535299e+07,23
2,Latin America & Caribbean,662185950,1.576633e+07,42
4,North America,381464223,1.271547e+08,3


In [104]:
region_summary_csv = gold_path / "gold_pipulation_by_region_testing.csv"
region_summary.to_csv(region_summary_csv, index=False)

In [105]:
top10 = gold_data.sort_values("population", ascending=False).head(10).assign(
    population_text = lambda df: (df["population"] / 1_000_000 ).round(1).astype(str) + "M"
)

top10_chart = px.bar(
    top10,
    x="Official_Name",
    y = "population",
    title="plot1",
    text="population_text"
)
top10_chart.update_layout(xaxis_tickangle=-45)

top10_chart.show()


In [106]:
region_chart = px.bar(
    region_summary,
    x="region_name",
    y = "total_population",
    title="plot2",
    text=(region_summary["total_population"] / 1_000_000_000 ).round(1).astype(str) + "B"

)


region_chart.update_layout(xaxis_tickangle=-45)

region_chart.show()